In [33]:
# import libraries
from statsbombpy import sb
import json
import pandas as pd
import matplotlib.pyplot as plt
from mplsoccer.pitch import Pitch
import seaborn as sns
import os
import re

In [3]:
# obtain list of matches for specific competition and season 
df_matches = sb.matches(competition_id=55, season_id=43)
df_matches

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,...,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
0,3788753,2021-06-16,15:00:00.000,Europe - UEFA Euro,2020,Finland,Russia,0,1,available,...,2021-09-22T16:39:05.697512,2,Group Stage,Saint-Petersburg Stadium,Danny Desmond Makkelie,Markku Kanerva,Stanislav Cherchesov,1.1.0,2,2
1,3788765,2021-06-20,18:00:00.000,Europe - UEFA Euro,2020,Switzerland,Turkey,3,1,available,...,2021-11-11T13:54:37.507376,3,Group Stage,Bakı Olimpiya Stadionu,Slavko Vinčić,Vladimir Petković,Şenol Güneş,1.1.0,2,2
2,3795107,2021-07-02,21:00:00.000,Europe - UEFA Euro,2020,Belgium,Italy,1,2,available,...,2021-09-23T00:02:51.495862,5,Quarter-finals,Allianz Arena,Slavko Vinčić,Roberto Martínez Montoliú,Roberto Mancini,1.1.0,2,2
3,3795221,2021-07-07,21:00:00.000,Europe - UEFA Euro,2020,England,Denmark,2,1,available,...,2021-09-22T22:33:37.494366,6,Semi-finals,Wembley Stadium,Danny Desmond Makkelie,Gareth Southgate,Kasper Hjulmand,1.1.0,2,2
4,3795506,2021-07-11,21:00:00.000,Europe - UEFA Euro,2020,Italy,England,1,1,available,...,2021-09-22T22:40:31.690550,7,Final,Wembley Stadium,Björn Kuipers,Roberto Mancini,Gareth Southgate,1.1.0,2,2
5,3794688,2021-06-29,18:00:00.000,Europe - UEFA Euro,2020,England,Germany,2,0,available,...,2021-09-22T23:46:25.457149,4,Round of 16,Wembley Stadium,Danny Desmond Makkelie,Gareth Southgate,Joachim Löw,1.1.0,2,2
6,3794692,2021-06-29,21:00:00.000,Europe - UEFA Euro,2020,Sweden,Ukraine,1,2,available,...,2021-09-22T23:38:19.483199,4,Round of 16,Hampden Park,Daniele Orsato,Jan Olof Andersson,Andrii Shevchenko,1.1.0,2,2
7,3794686,2021-06-28,18:00:00.000,Europe - UEFA Euro,2020,Croatia,Spain,3,5,available,...,2021-09-22T23:30:10.428563,4,Round of 16,Parken,Cüneyt Çakιr,Zlatko Dalić,Luis Enrique Martínez García,1.1.0,2,2
8,3794687,2021-06-27,21:00:00.000,Europe - UEFA Euro,2020,Belgium,Portugal,1,0,available,...,2021-09-22T16:40:48.020474,4,Round of 16,Estadio de La Cartuja,Felix Brych,Roberto Martínez Montoliú,Fernando Manuel Fernandes da Costa Santos,1.1.0,2,2
9,3794685,2021-06-26,21:00:00.000,Europe - UEFA Euro,2020,Italy,Austria,2,1,available,...,2021-09-22T16:40:41.310001,4,Round of 16,Wembley Stadium,Anthony Taylor,Roberto Mancini,Franco Foda,1.1.0,2,2


In [26]:
# get all event info
df_events = pd.read_csv('full_data_sorted.csv')

/var/folders/fk/svllg0550bj0tsxddmfx5f1w0000gn/T/ipykernel_63179/355599449.py:2: DtypeWarning: Columns (2,4,12,26,35,43,49,69,85,86,87,88,89,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111) have mixed types. Specify dtype option on import or set low_memory=False.
  df_events = pd.read_csv('full_data_sorted.csv')


In [27]:
df_events.columns.tolist()

['Unnamed: 0',
 'ball_receipt_outcome',
 'ball_recovery_offensive',
 'ball_recovery_recovery_failure',
 'block_deflection',
 'carry_end_location',
 'clearance_aerial_won',
 'clearance_body_part',
 'clearance_head',
 'clearance_left_foot',
 'clearance_right_foot',
 'counterpress',
 'dribble_nutmeg',
 'dribble_outcome',
 'dribble_overrun',
 'duel_outcome',
 'duel_type',
 'duration',
 'foul_committed_advantage',
 'foul_committed_card',
 'foul_won_advantage',
 'foul_won_defensive',
 'goalkeeper_body_part',
 'goalkeeper_end_location',
 'goalkeeper_outcome',
 'goalkeeper_position',
 'goalkeeper_punched_out',
 'goalkeeper_technique',
 'goalkeeper_type',
 'id',
 'index',
 'interception_outcome',
 'location',
 'match_id',
 'minute',
 'miscontrol_aerial_won',
 'off_camera',
 'out',
 'pass_aerial_won',
 'pass_angle',
 'pass_assisted_shot_id',
 'pass_body_part',
 'pass_cross',
 'pass_deflected',
 'pass_end_location',
 'pass_goal_assist',
 'pass_height',
 'pass_inswinging',
 'pass_length',
 'pass_n

In [28]:
df_throwIn = df_events.loc[df_events['play_pattern'] == 'From Throw In']
df_throwIn

,Unnamed: 0,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,shot_saved_to_post,shot_redirect,pass_straight,goalkeeper_shot_saved_off_target,shot_saved_off_target,shot_follows_dribble,dribble_no_touch,goalkeeper_penalty_saved_to_post,goalkeeper_success_in_play,player_off_permanent
12,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,1067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,2088,NaN,NaN,NaN,NaN,"[3.3, 12.7]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,1068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192565,2803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192566,4023,NaN,NaN,NaN,NaN,"[70.4, 28.6]",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192567,4633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192585,1419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
df_throwIn['timestamp']

12        00:00:12.163
13        00:00:14.093
14        00:00:14.093
15        00:00:16.420
16        00:00:18.242
              ...     
192565    00:13:58.761
192566    00:13:58.761
192567    00:13:59.242
192585    00:14:23.091
192586    00:14:25.224
Name: timestamp, Length: 42766, dtype: object

In [8]:
# df_throwIn_sort = df_throwIn.sort_values(by = "index")
# df_throwIn_sort

,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_right_foot,...,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,timestamp,type,under_pressure
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:12.163,Pass,NaN
1067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:14.093,Ball Receipt*,NaN
2088,NaN,NaN,NaN,NaN,"[3.3, 12.7]",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:14.093,Carry,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:16.420,Pass,NaN
1068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:00:18.242,Ball Receipt*,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:46:42.050,Ball Receipt*,NaN
2935,NaN,NaN,NaN,NaN,"[33.9, 49.2]",NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:46:42.050,Carry,True
3320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Turkey,00:46:42.655,Pressure,NaN
1050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,Italy,00:46:42.815,Pass,True


In [39]:
df_throwIn_new_index = df_throwIn.reset_index()
df_throwIn_final = df_throwIn_new_index.rename(columns={'level_0': 'full_index','Unnamed: 0': 'old_index'})
df_throwIn_final

,full_index,old_index,ball_receipt_outcome,ball_recovery_offensive,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,...,shot_saved_to_post,shot_redirect,pass_straight,goalkeeper_shot_saved_off_target,shot_saved_off_target,shot_follows_dribble,dribble_no_touch,goalkeeper_penalty_saved_to_post,goalkeeper_success_in_play,player_off_permanent
0,12,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13,1067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14,2088,NaN,NaN,NaN,NaN,"[3.3, 12.7]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,15,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,16,1068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42761,192565,2803,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42762,192566,4023,NaN,NaN,NaN,NaN,"[70.4, 28.6]",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42763,192567,4633,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42764,192585,1419,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
# create dictionary deeming each throw in a success or failure
# success defined as retaining possession for at least 7 seconds 

'''
from datetime import datetime

throw_in_outcomes = {}

current_possession = 0
looking_for_result = False

for row in df_throwIn_final.index:
    if df_throwIn_final["pass_type"][row] == "Throw-in" and looking_for_result == False:
        initial_time = df_throwIn_final["timestamp"][row]
        initial_dt_time = datetime.strptime(initial_time, '%H:%M:%S.%f')
        current_possession = df_throwIn_final["possession"][row]
        looking_for_result = True

        # print(current_possession)
    # print((df_throwIn_final["possession"][row]) > current_possession)
    # print(current_possession)
    # print(df_throwIn_final["possession"][row])
    # print("-")


    elif (looking_for_result == True) and ((df_throwIn_final["possession"][row]) > current_possession):
        end_time = df_throwIn_final["timestamp"][row]
        end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
        time_diff = end_dt_time - initial_dt_time
        if time_diff.total_seconds() >= 7:
            throw_in_outcomes[current_possession] = "Success"
            looking_for_result = False
        else:
            throw_in_outcomes[current_possession] = "Failure"
            looking_for_result = False
        if df_throwIn_final["pass_type"][row] == "Throw-in":
            initial_time = df_throwIn_final["timestamp"][row]
            initial_dt_time = datetime.strptime(initial_time, '%H:%M:%S.%f')
            current_possession = df_throwIn_final["possession"][row]
            looking_for_result = True

print(throw_in_outcomes)    
'''

{3: 'Success', 4: 'Success', 5: 'Success', 7: 'Success', 8: 'Success', 10: 'Success', 12: 'Success', 13: 'Success', 14: 'Success', 17: 'Success', 18: 'Success', 23: 'Success', 33: 'Success', 34: 'Success', 38: 'Success', 41: 'Success', 42: 'Success', 46: 'Success', 47: 'Success', 52: 'Success', 56: 'Success', 62: 'Success', 64: 'Success', 71: 'Success', 75: 'Success', 80: 'Success', 98: 'Success', 99: 'Success', 104: 'Failure', 115: 'Success', 116: 'Success', 129: 'Success', 133: 'Success', 137: 'Success', 144: 'Success', 145: 'Success', 147: 'Success', 151: 'Success', 156: 'Success', 165: 'Success', 166: 'Success', 167: 'Success', 168: 'Success', 170: 'Success'}


In [34]:
# assign directory
directory = 'data/three-sixty'

# list of filenames
file_list = []
 
# iterate over files in
# that directory
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    # checking if it is a file
    if os.path.isfile(f):
        f_number = re.split("[/.]", f)[2]
        file_list.append(f_number)

file_list

['3788742',
 '3788754',
 '3794690',
 '3794686',
 '3788758',
 '3788774',
 '3788762',
 '3795107',
 '3788763',
 '3788775',
 '3788759',
 '3794687',
 '3794691',
 '3788755',
 '3788743',
 '3788748',
 '3788772',
 '3788764',
 '3795506',
 '3788744',
 '3788752',
 '3788768',
 '3788769',
 '3788753',
 '3788745',
 '3788765',
 '3788773',
 '3788749',
 '3788766',
 '3788770',
 '3788750',
 '3788746',
 '3788747',
 '3788751',
 '3788771',
 '3788767',
 '3795109',
 '3795220',
 '3788756',
 '3794688',
 '3788760',
 '3788776',
 '3795187',
 '3794692',
 '3794685',
 '3788761',
 '3788741',
 '3794689',
 '3788757',
 '3795221',
 '3795108']

In [47]:
# create dictionary deeming each throw in a success or failure
# success defined as retaining possession for at least 7 seconds 

from datetime import datetime

throw_in_outcomes = {}

current_possession = 0
i = 0

throwin_count = 0

for match in file_list:
    throwin_row = 0
    looking_for_result = False
    elapsed_time = 0
    current_possession = 0
    i = 0

    initial_match_df = df_throwIn_final[df_throwIn_final["match_id"]== int(match)]
    match_df = initial_match_df.reset_index()

    while i < len(match_df):
        if (match_df.loc[i, "pass_type"] == "Throw-in") and (looking_for_result == False):
            throwin_count += 1
            throwin_row = i
            elapsed_time = 0
            start_time = match_df.loc[i, "timestamp"]
            start_dt_time = datetime.strptime(start_time, '%H:%M:%S.%f')
            current_possession = match_df.loc[i, "possession"]
            looking_for_result = True
            # print(throwin_row)
        
        elif looking_for_result == True:
            end_time = match_df.loc[i, "timestamp"]
            end_dt_time = datetime.strptime(end_time, '%H:%M:%S.%f')
            elapsed_time = elapsed_time + (end_dt_time - start_dt_time).total_seconds()
            start_dt_time = end_dt_time

            if elapsed_time >= 7:
                if match_df.loc[i, "possession"] == current_possession:
                    throw_in_outcomes[throwin_count] = [match_df.loc[throwin_row, "location"], "Success"]
                else:
                    throw_in_outcomes[throwin_count] = [match_df.loc[throwin_row, "location"], "Failure"]
                    i = throwin_row
                looking_for_result = False
        
        i = i + 1

throw_in_outcomes



{1: ['[20.8, 0.1]', 'Success'],
 2: ['[59.2, 0.1]', 'Success'],
 3: ['[112.2, 0.1]', 'Success'],
 4: ['[26.8, 80.0]', 'Success'],
 5: ['[7.7, 80.0]', 'Failure'],
 6: ['[110.9, 0.1]', 'Failure'],
 7: ['[92.9, 80.0]', 'Failure'],
 8: ['[102.2, 80.0]', 'Failure'],
 9: ['[58.5, 0.1]', 'Failure'],
 10: ['[66.9, 80.0]', 'Failure'],
 11: ['[70.9, 0.1]', 'Failure'],
 12: ['[81.1, 0.1]', 'Success'],
 13: ['[48.4, 0.1]', 'Success'],
 14: ['[49.7, 0.1]', 'Success'],
 15: ['[49.1, 80.0]', 'Success'],
 16: ['[98.2, 0.1]', 'Success'],
 17: ['[99.9, 80.0]', 'Failure'],
 18: ['[83.8, 80.0]', 'Success'],
 19: ['[98.6, 0.1]', 'Success'],
 20: ['[87.8, 0.1]', 'Success'],
 21: ['[44.5, 80.0]', 'Success'],
 22: ['[112.1, 0.1]', 'Success'],
 23: ['[40.4, 0.1]', 'Failure'],
 24: ['[29.5, 0.1]', 'Success'],
 25: ['[91.6, 0.1]', 'Failure'],
 26: ['[110.7, 0.1]', 'Failure'],
 27: ['[108.7, 0.1]', 'Success'],
 28: ['[26.1, 0.1]', 'Failure'],
 29: ['[72.5, 0.1]', 'Failure'],
 30: ['[56.7, 80.0]', 'Success'],
 31:

In [60]:
# linear regression
import numpy as np
from sklearn.linear_model import LinearRegression

list_of_x_coordinates = []
for throwin in throw_in_outcomes:
    coords = throw_in_outcomes[throwin][0]
    x_coord = coords.split(",")[0][1:]
    list_of_x_coordinates.append(float(x_coord))

list_of_outcomes = []
for throwin in throw_in_outcomes:
    outcome = throw_in_outcomes[throwin][1]
    if outcome == "Success":
        outcome_val = 1
    else:
        outcome_val = 0

    list_of_outcomes.append(outcome_val)

x = np.array(list_of_x_coordinates).reshape((-1, 1))
print(x)

y = np.array(list_of_outcomes)
print(y)


[[ 20.8]
 [ 59.2]
 [112.2]
 ...
 [106.4]
 [ 36.2]
 [ 54.8]]
[1 1 1 ... 1 1 0]


In [61]:
# modeling

model = LinearRegression()

model.fit(x, y)

LinearRegression()

In [63]:
r_sq = model.score(x, y)
print(f"coefficient of determination: {r_sq}")
print(f"intercept: {model.intercept_}")
print(f"slope: {model.coef_}")

coefficient of determination: 0.014712869011338503
intercept: 0.5429441047435098
slope: [0.00195444]


In [64]:
y_pred = model.predict(x)
print(f"predicted response:\n{y_pred}")

predicted response:
[0.58359654 0.6586472  0.76223275 ... 0.75089697 0.61369499 0.65004765]
